# Quasi-criticality in the Cortex

## Imports

In [ ]:
conda install -c conda-forge powerlaw


In [ ]:
'''Setup notebook environment -q flag suppresses output, if you want to see it, remove the -q flag'''
# %pip install -r requirements.txt -q
from utils.plotting_utils import *
from utils.data_utils import *
from utils.utils import simulate
from branching import BranchingNeurons
import os
import numpy as np
import matplotlib.pyplot as plt
import powerlaw
import pandas as pd

## Branching model

In [ ]:
if os.path.exists('data/branching_data.csv'):
    data = load_data('data/branching_data.csv')
else:
    kwargs = {
        'N': 2500,
        'max_neighbors': 15,
        'visual': False,
    }
    data = simulate(BranchingNeurons, n_runs=3, duration=10000, **kwargs)


In [ ]:
def get_max__length(data):
    max_length = 0
    for j in range(len(data)):
        for i in range(len(data[j])):
            if len(data[j][i]) > max_length:
                max_length = len(data[j][i])
    return max_length

def pad_data_to_max_length(data, max_length):
    for j in range(len(data)):
        for i in range(len(data[j])):
            if len(data[j][i]) < max_length:
                data[j][i] = np.pad(data[j][i], (0, max_length - len(data[j][i])), 'constant', constant_values=(0, 0))
    return data

homogenous_data = np.array(pad_data_to_max_length(data, get_max__length(data)))

In [ ]:
duration_data = homogenous_data[:,0,:]
size_data = homogenous_data[:,1,:]

In [ ]:
flattened = size_data.flatten()
plt.hist(flattened, histtype='step', density=True)
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:

# duration_data = duration_data.sort(axis=1)
# size_data = size_data.sort(axis=1)
# mean_duration = np.mean(duration_data, axis=0)
# mean_size = np.mean(size_data, axis=0)
mean_duration = duration_data[0]
mean_size = size_data[0]

plt.scatter(mean_duration, mean_size)
plt.xscale('log')
plt.yscale('log')
plt.show()

## BTW-like model

### Settings of parameters and writing data

In [ ]:
settings1 = [
    {"name": "round_spiral", "params": {"height": 4, "refractory_period": 5, "probability_of_spontaneous_activity": 0.02, "max_distance": 3, "random_connection": False}},
    {"name": "pulse_wave", "params": {"height": 5, "refractory_period": 4, "probability_of_spontaneous_activity": 0.03, "max_distance": 3, "random_connection": False}},
    {"name": "synchronous", "params": {"height": 3, "refractory_period": 5, "probability_of_spontaneous_activity": 0.015, "max_distance": 2.5, "random_connection": True}},
    {"name": "oscillatory", "params": {"height": 2, "refractory_period": 4, "probability_of_spontaneous_activity": 0.02, "max_distance": 3, "random_connection": False}},
    {"name": "repeating", "params": {"height": 2, "refractory_period": 4, "probability_of_spontaneous_activity": 0.02, "max_distance": 3, "random_connection": True}},
    {"name": "random", "params": {"height": 5, "refractory_period": 5, "probability_of_spontaneous_activity": 0.02, "max_distance": 3, "random_connection": False}}]

In [ ]:
# Data collection
for setting in settings1:
    btw = BTW(grid_size=[50, 50], **setting['params'])
    btw.run(10000)
    path = f"data/spikes_btw_{setting['name']}.csv"
    btw.write_data(path)

In [ ]:
# Plot avg_spike_density vs. m
paths = [f"data/spikes_btw_{setting['name']}.csv" for setting in settings1]
size = 50
spike_density_plot(paths, size)

In [ ]:
settings2 = [{"name": f"ref{ref}thresh{thresh}p{p}r{r}", 
            "params": {"height": thresh, 
                        "refractory_period": ref, 
                        "probability_of_spontaneous_activity": p, 
                        "max_distance": 3, 
                        "random_connection": r}}
            for ref in range(1, 8) for thresh in range(1, 8) for p in [0.015, 0.02, 0.025] for r in [False, True]]

In [ ]:
# Data collection
for setting in settings2:
    btw = BTW(grid_size=[50, 50], **setting['params'])
    btw.init_grid("random", 4)
    btw.run(10000)
    path = f"data/spikes_btw_ref_thresh/spikes_btw_{setting['name']}.csv"
    btw.write_data(path)

### Avg_spike_density vs. m(Considering refractory period)

In [ ]:
# Plot avg_spike_density vs. m
paths = [f"data/spikes_btw_ref_thresh/spikes_btw_{setting['name']}.csv" for setting in settings2]
size = 50
# Get The refractory_periods list from the seeting2 data

refractory_periods = [setting['params']['refractory_period'] for setting in settings2]

ref_spike_density_plot(paths, size, refractory_periods)


### Avg_spike_density vs. m(Not considering refractory period)

In [ ]:
paths = [f"data/spikes_btw_ref_thresh/spikes_btw_{setting['name']}.csv" for setting in settings2]
size = 50

spike_density_plot(paths, size)


### Avalanche size/duration: historical


In [ ]:
settings3 = [{"name": f"ref{ref}thresh{4}p{p}r{r}", 
            "params": {"height": 4, 
                        "refractory_period": ref, 
                        "probability_of_spontaneous_activity": p, 
                        "max_distance": 3, 
                        "random_connection": r}}
            for ref in range(4, 7) for p in [0.02, 0.025] for r in [False,True]]
paths = [f"data/spikes_btw_ref_thresh/spikes_btw_{setting['name']}.csv" for setting in settings3]
powerlaw_avalanche_plots(paths, 'histogram', 0.1)

### Avalanche size/duration: scatter

In [ ]:
settings3 = [{"name": f"ref{ref}thresh{4}p{p}r{r}", 
            "params": {"height": 4, 
                        "refractory_period": ref, 
                        "probability_of_spontaneous_activity": p, 
                        "max_distance": 3, 
                        "random_connection": r}}
            for ref in range(4, 7) for p in [0.02, 0.025] for r in [False, True]]
paths = [f"data/spikes_btw_ref_thresh/spikes_btw_{setting['name']}.csv" for setting in settings3]
powerlaw_avalanche_plots(paths, 'scatter', 0.1)

### Avalanche size/duration: line

In [ ]:
settings3 = [{"name": f"ref{ref}thresh{t}p{p}r{r}", 
            "params": {"height": 4, 
                        "refractory_period": ref, 
                        "threshold": t,
                        "probability_of_spontaneous_activity": p, 
                        "max_distance": 3, 
                        "random_connection": r}}
            for ref in range(1, 8) for t in range(1,8) for p in [0.015,0.02, 0.025] for r in [False, True]]
paths = [f"data/spikes_btw_ref_thresh/spikes_btw_{setting['name']}.csv" for setting in settings3]
powerlaw_avalanche_plots(paths, 'plot', 0.1)

### Avalanche size/duration: fitting

In [ ]:
settings3 = [{"name": f"ref{ref}thresh{4}p{p}r{r}", 
            "params": {"height": 4, 
                        "refractory_period": ref, 
                        "probability_of_spontaneous_activity": p, 
                        "max_distance": 3, 
                        "random_connection": r}}
            for ref in range(4, 8) for p in [0.015, 0.02] for r in [False, True]]
paths = [f"data/spikes_btw_ref_thresh/spikes_btw_{setting['name']}.csv" for setting in settings3]
powerlaw_avalanche_plots(paths, 'fit', 0.1)

**Draft**

In [ ]:
results = load_data_csv("data/test_raster")
results.head()

In [ ]:
transmission_df = raster_to_transmission(results)
transmission_df.head()

In [ ]:
param_cols = ['max_distance', 'grid_size', 'random_connection', 'refractory_period', 'height', 'probability_of_spontaneous_activity']
results = results.drop(columns=param_cols)

In [ ]:
for index, row in results.iterrows():
    if index == 1:
        a = [int(col) for col in results.columns if row[col] != 0]
        print(np.sum(a))
        print(len(row))